In [1]:
project_name='anytown'

import os
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json 
import pandas as pd

#add ../pybeme to the path
import sys
sys.path.append('../pybeme')
sys.path.append('..')

from pybeme.beme_experiment import load_experiment_results, extract_dataframe
from pybeme.reference_set import naive_pareto_front

Naive Pareto Front: [[1 2]]


In [2]:
# Helper functions to run the analysis
# Load the settings file of the optimization problem and isolate the problem settings
def filter_problem_settings(settings_file, folder=""):
    with open(settings_file, 'r') as file:
        settings = json.load(file)

    with open(f'.tmp/bemesim__settings.json', 'w') as file:
        sett2write = settings['Typical configuration']['UDP']
        if len(folder)>0:
            sett2write['Lookup paths']=folder
        json.dump(settings['Typical configuration']['UDP'], file)
    
# Create a file with the individual that you want to save
def save_individual_json(individual):
    id = individual['id']
    with open(f'.tmp/bemesim__{id}__dv.json', 'w') as file:
        individual["Print"]="" # add this to avoid error 
        json.dump(individual, file)

# delete all this temporary files :) 
def clear_space():
    os.remove('.tmp')


In [3]:
# Upload the results.

folder_path=os.path.join(os.getcwd(), '..')

print(f"Using folder '{folder_path}' for the output results")

rehab_siew = load_experiment_results(os.path.join(folder_path, 'data',
                        'bemeopt__nsga2__anytown_rehab_f1__exp07', 'output', 'bemeopt__nsga2__anytown_rehab_f1__exp07__exp.json') )

mixed = load_experiment_results(os.path.join(folder_path,'data',
                        'bemeopt__nsga2__anytown_rehab_f1__exp07', 'output','bemeopt__nsga2__anytown_rehab_f1__exp07__exp.json') )

rehab_fpow = load_experiment_results(os.path.join(folder_path, 'data',
                        'bemeopt__nsga2__anytown_rehab_f1__exp06', 'output', 'bemeopt__nsga2__anytown_rehab_f1__exp06__exp.json') )

rehab_median = load_experiment_results(os.path.join(folder_path, 'data',
                        'bemeopt__nsga2__anytown_rehab_f1__exp05', 'output', 'bemeopt__nsga2__anytown_rehab_f1__exp05__exp.json') )

twoph = load_experiment_results(os.path.join(
  os.path.realpath(os.path.join(folder_path, 'data', 'bemeopt__nsga2__anytown_2ph_f1__nsga2inside')), 
  'output', 'bemeopt__nsga2__anytown_2ph_f1__nsga2inside__exp.json') )

exps = [mixed, rehab_siew, rehab_fpow, rehab_median, twoph]

Using folder '/Users/zannads/repos/BevarMejo/visual/..' for the output results


In [4]:
# Setup some helper functions to plot the results and the properties of the plot
#           blue night blue grey grey orange
colors = ['#29378A', '#808080', '#74BDA7','#02B9EA', '#F78132']
markers = ['circle', 'circle', 'square', 'diamond', 'circle']
msize = 15
names = ['Integrated', 'Pure design', 'Pure design', 'Pure design', 'Two-phase']
groupnames_pured = ['Siew et al.', 'Full power', 'Median']
namesind = ['Best', 'All runs']
figspace_in = [2.5, 1.5] # in inches the space reserved for the figure in the word document
dpi_mac = 227
slide_size = [13.33333333, 7.5] # in inches, the size of the slide in the presentation or screen
dpi_720p = 720/slide_size[1]
dpi_1080p = 1080/slide_size[1]
dpi = dpi_1080p
xlims = [-.5e6, 20.5e6]
ylims = [-0.05, 0.605]
xaxis_title_text='Cost [$]'
yaxis_title_text='Reliability Index [-]'
title_font_size=45
legend_font_size=35
def_font_size=30
def fix_layout(a_fig: go.Figure, a_title: str) -> None:
    a_fig.update_layout(title=dict(
                            text=a_title,
                            xanchor='center',
                            x=0.5,
                            yanchor='top',
                            y=1,
                            font_size=title_font_size
                        ),
                        plot_bgcolor='white',
                        paper_bgcolor='white',
                        xaxis=dict(
                            title=xaxis_title_text,
                            range=xlims,
                            automargin=True,
                            showline=True,
                            showgrid=True,
                            linewidth=1,
                            linecolor='grey',
                            zerolinecolor='black',
                            gridcolor='lightgrey'
                        ),
                        yaxis=dict(
                            title=yaxis_title_text,
                            range=ylims,
                            automargin=True,
                            showline=True,
                            showgrid=True,
                            linewidth=1,
                            linecolor='grey',
                            zerolinecolor='black',
                            gridcolor='lightgrey'
                        ),
                        width=figspace_in[0]*dpi,
                        height=figspace_in[1]*dpi,
                        font=dict(
                            family="Lato",
                            color="black",
                            size=def_font_size
                        ),
                        margin=dict(
                            l=10,
                            r=10,
                            b=10,
                            t=50,
                            pad=0
                        ),
                        showlegend=True,
                        legend=dict(
                            orientation="v",
                            xanchor="left",
                            x=0.03,  
                            yanchor="top",
                            y=0.9,  
                            itemsizing='trace',  # To ensure items in legend keep the same size
                            traceorder="normal",
                            bgcolor="White",  # Background color
                            bordercolor="Black",  # Border color
                            borderwidth=1,  # Border width
                            groupclick="toggleitem",
                            itemclick="toggleothers",
                            itemdoubleclick="toggle",
                            tracegroupgap=100,
                            font_size=legend_font_size
                        )
)


In [5]:
title='Average Daily Pattern'
t=np.array(np.arange(0, 24, 1)).reshape(-1, 1)
t=np.hstack([t,t]).flatten()
p=np.array([1.0,1.0,1.0,0.9,0.9,0.9,0.7,0.7,0.7,0.6,0.6,0.6,1.2,1.2,1.2,1.3,1.3,1.3,1.2,1.2,1.2,1.1,1.1,1.1]).reshape(-1, 1)
p=np.hstack([p,p]).flatten()
fig=go.Figure()
fig.add_trace(go.Scatter(x=t, y=p, mode='lines', showlegend=False,name='Average Daily Pattern', line=dict(color='#2683C6', width=3)))
fix_layout(fig, title)
fig.update_layout(width=2.5*dpi, height=0.75*dpi, title_text=None,font_size=10)
tlab=['18:00', '23:00', '04:00', '09:00', '14:00']
fig.update_xaxes(title=None, range=[0, 24], showline=True, showgrid=True, linewidth=1, linecolor='grey', zerolinecolor='black', gridcolor='lightgrey', tickvals=np.array(np.arange(0,24,5)), ticktext=tlab)
fig.update_yaxes(title=None, range=[0.5, 1.5], showline=True, showgrid=True, linewidth=1, linecolor='grey', zerolinecolor='black', gridcolor='lightgrey')

fig.show()

### Pareto front of all formulations
Final pareto front (non dominated solutions of the final iteration of the optimization process) of each formulation (see Legend).

In [6]:
title='Pareto fronts of all formulations'
figspace_in = [10, 6] 
fig =go.Figure()
for e in [1, 0, 4]:
    # for each island in islands and individual in individual extract the fitness-vector and put it in a numpy array
    fitness = np.array([ind['fitness-vector'] for island in exps[e]['archipelago']['islands'] for ind in island['generations'][-1]['individuals']])

    # I want full color the best pareto front of each solution and a lighter color for the rest, which are still pareto fronts but for the individual islands
    # Also, I need to make transparent solutions in the best pareto front but that are not feasible (i.e. reliability index < 0)
    pf = naive_pareto_front(fitness, f__indexes=True)
    pf = pf[fitness[pf,1] <= -0.1] # only feasible solutions (I should do a simulation but this will do)
    
    fig.add_trace(go.Scatter(x=fitness[pf,0], y=-fitness[pf,1], mode='markers', marker=dict(size=msize, symbol=markers[e], color=colors[e]),  
                             showlegend=True, name=names[e],
                             customdata=np.array(pf, dtype=str),
                             hovertemplate='Cost: %{x:2.2f} <br> Reliability Index: %{y:.2f} <extra>%{customdata}</extra>'
                             ) )
    
fix_layout(fig, title)
fig.show()

In [7]:
#same again but smaller for another slide
figspace_in=[7,5]
fix_layout(fig, title)
fig.show()

In [8]:
title='Pareto fronts of all formulations (Tanks detail)'
fig =go.Figure()
for e in [1, 0, 4]:
    # for each island in islands and individual in individual extract the fitness-vector and put it in a numpy array
    fitness = np.array([ind['fitness-vector'] for island in exps[e]['archipelago']['islands'] for ind in island['generations'][-1]['individuals']])
    dvs = np.array([ind['decision-vector'] for island in exps[e]['archipelago']['islands'] for ind in island['generations'][-1]['individuals']])

    # I want full color the best pareto front of each solution and a lighter color for the rest, which are still pareto fronts but for the individual islands
    # Also, I need to make transparent solutions in the best pareto front but that are not feasible (i.e. reliability index < 0)
    pf = naive_pareto_front(fitness, f__indexes=True)
    pf = pf[fitness[pf,1] <= -0.1] # only feasible solutions (I should do a simulation but this will do)

    no_tanks=pf[(dvs[pf,-4]==0) & (dvs[pf,-2]==0)]
    two_tanks=pf[(dvs[pf,-4]>0) & (dvs[pf,-2]>0) & (dvs[pf,-4]!=dvs[pf,-2])]
    one_tank=[p for p in pf if p not in no_tanks and p not in two_tanks]

    for t, nt in enumerate([no_tanks, one_tank, two_tanks]):
        mrk=dict(size=msize, symbol=markers[e], color=colors[e],line=dict(width=4, color=colors[e]))
        if t==0:
            mrk['symbol']=markers[e]+'-open'
            sl=False
        elif t==1:
            mrk['symbol']=markers[e]
            sl=True
        else:
            mrk['symbol']=markers[e]+'-dot'
            mrk['line']['color']='black'
            sl=False

        fig.add_trace(go.Scatter(x=fitness[nt,0], y=-fitness[nt,1], mode='markers', marker=mrk,  
                             showlegend=sl, name=names[e], legendgroup=names[e],
                             customdata=np.array(nt, dtype=str),
                             hovertemplate='Cost: %{x:2.2f} <br> Reliability Index: %{y:.2f} <extra>%{customdata}</extra>'
                             ) )

fig.add_trace(go.Scatter(x=[-1000000], y=[-100000], mode='markers', marker=dict(size=msize, symbol='circle-open', color='black', line=dict(width=4, color='black')), name='No tanks' ) )
fig.add_trace(go.Scatter(x=[-1000000], y=[-100000], mode='markers', marker=dict(size=msize, symbol='circle',      color='black', line=dict(width=4, color='black')), name='One tank' ) )
fig.add_trace(go.Scatter(x=[-1000000], y=[-100000], mode='markers', marker=dict(size=msize, symbol='circle-dot',  color='white', line=dict(width=4, color='black')), name='Two tanks' ) )
    
fix_layout(fig, title)
fig.show()

In [9]:
title='Pareto fronts of all formulations (Electricity detail)'
# I need an helper function to get the design cost of a solution so that I can use this info in the PF
def get_design_cost(dv: list) -> float:
    dv_expipes=dv[0:70]
    dv_newpipes=dv[70:76]
    dv_tanks=dv[-4:]

    dc=0
    avail_d=[6,8,10,12,14,16,18,20,24,30]
    expipes_l=[6000 for i in range(35)]
    expipes_l[0]=12000
    expipes_l[1]=12000
    expipes_l[2]=12000
    expipes_l[3]=100
    expipes_l[5]=9000
    expipes_l[6]=9000
    expipes_l[9]=12000
    expipes_l[11]=12000
    expipes_l[-2]=101
    expipes_l[-9]=101
    expipes_d=[12,12,16,30,10,10,12,10,10,8,10,8,10,8,8,8,8,10,8,10,8,12,12,10,12,10,12,10,10,8,12,10,8,12,10]
    for p in range(35):
        # find the diameter of the pipe in avail_d and assign the index
        expipes_d[p]=avail_d.index(expipes_d[p])
    
    expipes_city=[False,True,True,True,False,False,False,False,False,
                  False,False,False,False,False,False, #ID 10to20 
                  False,False,False,False,False,True,True,True,True, #ID 21to30
                  True,True,True,True,True,False,True,True,False,False, #ID 31to40
                  True]
    expipes_cl_d_city= [26.2, 27.8, 34.1, 41.4, 50.2,58.5,66.2,76.8,109.2,142.5]
    expipes_cl_d_sub=  [14.2, 19.8, 25.1, 32.4, 40.2,48.5,57.2,66.8,85.5,116.1]
    expipes_cl_l_city= [17.0, 17.0, 17.0, 17.0, 18.2,19.8,21.6,23.5,30.1,41.3]
    expipes_cl_l_sub=  [12,   12,   12,   13,   14.2,15.5,17.1,20.2,1000000000,10000000]
    for p in range(35):
        a=int(dv_expipes[p*2])
        d=int(dv_expipes[p*2+1])
        if dv_expipes[p]==1:
            if expipes_city[p]==True:
                dc+=expipes_l[p]*expipes_cl_l_city[int(expipes_d[p])]
            else:
                dc+=expipes_l[p]*expipes_cl_l_sub[int(expipes_d[p])]
        elif dv_expipes[p]==2:
            if expipes_city[p]==True:
                dc+=expipes_l[p]*expipes_cl_d_city[d]
            else:
                dc+=expipes_l[p]*expipes_cl_d_sub[d]
        
    newpipes_l=[6000,6000,6000,6000,6000,9000]
    newpipes_cl_d=[12.8,17.8,22.5,29.2,36.2,43.6,51.5,60.1,77.0,105.0]
    for p in range(6):
        dc+=newpipes_l[p]*newpipes_cl_d[int(dv_newpipes[p])]

    tanks_cv=[115000,145000,325000,425000,600000]
    if dv_tanks[0]>0:
        dc+=tanks_cv[int(dv_tanks[1])]
    if dv_tanks[2]>0 and dv_tanks[2]!=dv_tanks[0]:
        dc+=tanks_cv[int(dv_tanks[3])]
    
    return dc

fig =go.Figure()
max_ope_cost=17488932.511617582 #got from a run of the code
min_ope_cost=2274159.11813082
for e in [1, 0, 4]:
    # for each island in islands and individual in individual extract the fitness-vector and put it in a numpy array
    fitness = np.array([ind['fitness-vector'] for island in exps[e]['archipelago']['islands'] for ind in island['generations'][-1]['individuals']])
    dvs = np.array([ind['decision-vector'] for island in exps[e]['archipelago']['islands'] for ind in island['generations'][-1]['individuals']])

    # I want full color the best pareto front of each solution and a lighter color for the rest, which are still pareto fronts but for the individual islands
    # Also, I need to make transparent solutions in the best pareto front but that are not feasible (i.e. reliability index < 0)
    pf = naive_pareto_front(fitness, f__indexes=True)
    pf = pf[fitness[pf,1] <= -0.1] # only feasible solutions (I should do a simulation but this will do)

    ddv_c=np.array([get_design_cost(dv) for dv in dvs[pf]])
    #max_ope_cost=max(max_ope_cost, np.max(fitness[pf,0]-ddv_c))
    #min_ope_cost=min(min_ope_cost, np.min(fitness[pf,0]-ddv_c))

    norm_ope_costs=((fitness[pf,0]-ddv_c)-min_ope_cost)/(max_ope_cost-min_ope_cost)
    mrk=dict(size=[msize+2*msize*noc for noc in norm_ope_costs], symbol=markers[e], color=colors[e], line=dict(width=2, color=colors[e]))
    
    fig.add_trace(go.Scatter(x=fitness[pf,0], y=-fitness[pf,1], mode='markers', marker=mrk,  
                            showlegend=True, name=names[e], legendgroup=names[e],
                            customdata=[[fitness[pf[i],0]/1e6, pf[i], (fitness[pf[i],0]/1e6-ddv_c[i]/1e6)] for i in range(len(pf))],
                            hovertemplate='Cost: %{customdata[0]:.2f}M $<br> Reliability Index: %{y:.2f} <extra>%{customdata[1]}<br>Ope. cost: %{customdata[2]:.2f}M $</extra>'
                            ) )
#print(max_ope_cost, min_ope_cost)

# add two fake points to the legend to show the ope cost in gray outisde of the limts and in another legend to show it in a second legend
fig.add_trace(go.Scatter(x=[-1000000], y=[-1000000], mode='markers', marker=dict(size=msize, color='black'), showlegend=True, name='Low Ope. cost', legendgroup='Ope. cost', hoverinfo='skip'))
fig.add_trace(go.Scatter(x=[-1000000], y=[-1000000], mode='markers', marker=dict(size=3*msize, color='black'), showlegend=True, name='High Ope. cost', legendgroup='Ope. cost', hoverinfo='skip'))
    
fix_layout(fig, title)
fig.update_layout(legend_itemsizing='trace')

fig.show()

Clearly, the joint design and operations approaches (Integrated and Two-phase) outperform the pure design approach:
- +150% and +100% at 10M $
- +60% and + 50% at 15M $

### Pareto front of the integrated formulation (detail)

In [10]:
title='Pareto fronts of the pure design formulation (detail)'
fig = go.Figure()

for e in [1,2,3]:
    # for each island in islands and individual in individual extract the fitness-vector and put it in a numpy array
    fitness = np.array([ind['fitness-vector'] for island in exps[e]['archipelago']['islands'] for ind in island['generations'][-1]['individuals']])
    dvs = np.array([ind['decision-vector'] for island in exps[e]['archipelago']['islands'] for ind in island['generations'][-1]['individuals']])

    # I want full color the best pareto front of each solution and a lighter color for the rest, which are still pareto fronts but for the individual islands
    # Also, I need to make transparent solutions in the best pareto front but that are not feasible (i.e. reliability index < 0)
    pf = naive_pareto_front(fitness, f__indexes=True)
    pf = pf[fitness[pf,1] <= -0.1] # only feasible solutions (I should do a simulation but this will do)
    npf=[p for p in range(len(fitness)) if p not in pf]

    mrk=dict(size=msize, symbol=markers[e], color=colors[e], line=dict(width=2, color=colors[e]))
    
    fig.add_trace(go.Scatter(x=fitness[pf,0], y=-fitness[pf,1], mode='markers', marker=mrk,  
                            showlegend=True, name='PD-Ope: '+groupnames_pured[e-1], legendgroup=names[e]+'-'+groupnames_pured[e-1],
                            customdata=[[fitness[pf[i],0]/1e6, pf[i]] for i in range(len(pf))],
                            hovertemplate='Cost: %{customdata[0]:.2f}M $<br> Reliability Index: %{y:.2f} <extra>%{customdata[1]}</extra>'
                            ) )
    mrk['symbol']=markers[e]+'-open'
    #make color from hex to rgba opaque at 0.2
    mrk['color']='rgba({},{},{},0.2)'.format(int(colors[e].lstrip('#')[0:2], 16), int(colors[e].lstrip('#')[2:4], 16), int(colors[e].lstrip('#')[4:6], 16))
    fig.add_trace(go.Scatter(x=fitness[npf,0], y=-fitness[npf,1], mode='markers', marker=mrk,  
                            showlegend=True, name='PD-Ope: '+groupnames_pured[e-1]+' (All)', legendgroup=names[e]+'-'+groupnames_pured[e-1],
                            customdata=[[fitness[npf[i],0]/1e6, npf[i]] for i in range(len(npf))],
                            hovertemplate='Cost: %{customdata[0]:.2f}M $<br> Reliability Index: %{y:.2f} <extra>%{customdata[1]}</extra>'
                            ) )
    
    
    
fix_layout(fig, title)
fig.update_layout(legend_y=0.99)
fig.show()

In [11]:
title='Pareto fronts of the pure design formulation (detail)'
fig = go.Figure()
e=0
# for each island in islands and individual in individual extract the fitness-vector and put it in a numpy array
fitness = np.array([ind['fitness-vector'] for island in exps[e]['archipelago']['islands'] for ind in island['generations'][-1]['individuals']])
dvs = np.array([ind['decision-vector'] for island in exps[e]['archipelago']['islands'] for ind in island['generations'][-1]['individuals']])

# I want full color the best pareto front of each solution and a lighter color for the rest, which are still pareto fronts but for the individual islands
# Also, I need to make transparent solutions in the best pareto front but that are not feasible (i.e. reliability index < 0)
pf = naive_pareto_front(fitness, f__indexes=True)
pf = pf[fitness[pf,1] <= -0.1] # only feasible solutions (I should do a simulation but this will do)
npf=[p for p in range(len(fitness)) if p not in pf]

mrk=dict(size=msize, symbol=markers[e], color=colors[e], line=dict(width=2, color=colors[e]))

fig.add_trace(go.Scatter(x=fitness[pf,0], y=-fitness[pf,1], mode='markers', marker=mrk,  
                        showlegend=True, name=names[e], legendgroup=names[e],
                        customdata=[[fitness[pf[i],0]/1e6, pf[i]] for i in range(len(pf))],
                        hovertemplate='Cost: %{customdata[0]:.2f}M $<br> Reliability Index: %{y:.2f} <extra>%{customdata[1]}</extra>'
                        ) )
mrk['symbol']=markers[e]+'-open'
#make color from hex to rgba opaque at 0.2
mrk['color']='rgba({},{},{},0.2)'.format(int(colors[e].lstrip('#')[0:2], 16), int(colors[e].lstrip('#')[2:4], 16), int(colors[e].lstrip('#')[4:6], 16))
fig.add_trace(go.Scatter(x=fitness[npf,0], y=-fitness[npf,1], mode='markers', marker=mrk,  
                        showlegend=True, name=names[e]+' (All)', legendgroup=names[e],
                        customdata=[[fitness[npf[i],0]/1e6, npf[i]] for i in range(len(npf))],
                        hovertemplate='Cost: %{customdata[0]:.2f}M $<br> Reliability Index: %{y:.2f} <extra>%{customdata[1]}</extra>'
                        ) )
    
fix_layout(fig, title)
fig.show()

In [12]:
title='Median pattern of the integrated opt. results'
fig = go.Figure()
figspace_in=[4.5,2]
e=0
# for each island in islands and individual in individual extract the fitness-vector and put it in a numpy array
fitness = np.array([ind['fitness-vector'] for island in exps[e]['archipelago']['islands'] for ind in island['generations'][-1]['individuals']])
dvs = np.array([ind['decision-vector'] for island in exps[e]['archipelago']['islands'] for ind in island['generations'][-1]['individuals']])

# I want full color the best pareto front of each solution and a lighter color for the rest, which are still pareto fronts but for the individual islands
# Also, I need to make transparent solutions in the best pareto front but that are not feasible (i.e. reliability index < 0)
pf = naive_pareto_front(fitness, f__indexes=True)
pf = pf[fitness[pf,1] <= -0.1] # only feasible solutions (I should do a simulation but this will do)
npf=[p for p in range(len(fitness)) if p not in pf]

dvs_ope_int=np.median(dvs[pf, 76:100], axis=0).reshape(-1,1)
dvs_ope_int=np.concatenate([dvs_ope_int,dvs_ope_int], axis=1).flatten()
dvs_ope_siew = np.array([3,2,2,2,2,2,2,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3]).reshape(-1,1)
dvs_ope_siew=np.concatenate([dvs_ope_siew,dvs_ope_siew], axis=1).flatten()
t=np.array(np.arange(0, 24, 1)).reshape(-1, 1)
t=np.hstack([t,t+1]).flatten()
fig.add_trace(go.Scatter(x=t, y=dvs_ope_int, mode='lines', showlegend=True, name='Integrated', line=dict(color=colors[0], width=3)))
fig.add_trace(go.Scatter(x=t, y=dvs_ope_siew, mode='lines', showlegend=True, name='Ope. from Siew et al.', line=dict(color=colors[1], width=3)))
fix_layout(fig, None)
fig.update_layout(font_size=12)
fig.update_xaxes(title=dict(text='Time of the day', font_size=14), range=[0, 24], tickvals=np.array(np.arange(0,24,5)), ticktext=['18:00', '23:00', '04:00', '09:00', '14:00'])
fig.update_yaxes(title=dict(text='# pumps', font_size=14), range=[-0.15, 3.15], tickvals=[0,1,2,3], ticktext=['0','1','2','3'])
fig.update_layout(legend_font_size=12, legend_x=0.2, legend_y=0.99)

In [13]:
title='Pareto fronts of the pure design formulation (detail)'
fig = go.Figure()
figspace_in=[7,5]
e=4
# for each island in islands and individual in individual extract the fitness-vector and put it in a numpy array
fitness = np.array([ind['fitness-vector'] for island in exps[e]['archipelago']['islands'] for ind in island['generations'][-1]['individuals']])
dvs = np.array([ind['decision-vector'] for island in exps[e]['archipelago']['islands'] for ind in island['generations'][-1]['individuals']])

# I want full color the best pareto front of each solution and a lighter color for the rest, which are still pareto fronts but for the individual islands
# Also, I need to make transparent solutions in the best pareto front but that are not feasible (i.e. reliability index < 0)
pf = naive_pareto_front(fitness, f__indexes=True)
pf = pf[fitness[pf,1] <= -0.1] # only feasible solutions (I should do a simulation but this will do)
npf=[p for p in range(len(fitness)) if p not in pf]

mrk=dict(size=msize, symbol=markers[e], color=colors[e], line=dict(width=2, color=colors[e]))

fig.add_trace(go.Scatter(x=fitness[pf,0], y=-fitness[pf,1], mode='markers', marker=mrk,  
                        showlegend=True, name=names[e], legendgroup=names[e],
                        customdata=[[fitness[pf[i],0]/1e6, pf[i]] for i in range(len(pf))],
                        hovertemplate='Cost: %{customdata[0]:.2f}M $<br> Reliability Index: %{y:.2f} <extra>%{customdata[1]}</extra>'
                        ) )
mrk['symbol']=markers[e]+'-open'
#make color from hex to rgba opaque at 0.2
mrk['color']='rgba({},{},{},0.2)'.format(int(colors[e].lstrip('#')[0:2], 16), int(colors[e].lstrip('#')[2:4], 16), int(colors[e].lstrip('#')[4:6], 16))
fig.add_trace(go.Scatter(x=fitness[npf,0], y=-fitness[npf,1], mode='markers', marker=mrk,  
                        showlegend=True, name=names[e]+' (All)', legendgroup=names[e],
                        customdata=[[fitness[npf[i],0]/1e6, npf[i]] for i in range(len(npf))],
                        hovertemplate='Cost: %{customdata[0]:.2f}M $<br> Reliability Index: %{y:.2f} <extra>%{customdata[1]}</extra>'
                        ) )
    
fix_layout(fig, title)
fig.show()

In [14]:
# Prepare workspace 
import wntr

In [15]:
# I need a way to click on a point and see the individual that generated that point
import subprocess
beme_folder='../'
e = 0 # experiment to use
individuals_I_care = [401, 1075, 1017]
all_individuals = [ind for island in exps[e]['archipelago']['islands'] for ind in island['generations'][-1]['individuals'] ]

filter_problem_settings(os.path.join(exps[e]['folder'], 'bemeopt__settings.json'),
                        folder=exps[e]['folder'])

for individual in individuals_I_care:
    individual_id = all_individuals[individual]['id']
    save_individual_json(all_individuals[individual])

    # run the simulation of beme to save the inp file from shell
    command=f'{beme_folder}build-vs-code/cli/beme-sim .tmp/bemesim__settings.json .tmp/bemesim__{individual_id}__dv.json --saveinp'
    subprocess.run(command, shell=True, check=True)

subprocess.run('mv *.inp .tmp/', shell=True, check=True)

  Page 1                                    Thu Aug 29 10:50:04 2024

  ******************************************************************
  *                           E P A N E T                          *
  *                   Hydraulic and Water Quality                  *
  *                   Analysis for Pipe Networks                   *
  *                         Version 2.3                            *
  ******************************************************************
  EPANET project deleted


An error happend while evaluating the decision vector:
vector


CalledProcessError: Command '../build-vs-code/cli/beme-sim .tmp/bemesim__settings.json .tmp/bemesim__17033784840247481730__dv.json --saveinp' returned non-zero exit status 1.

In [ ]:
results_int = {}
for individual in individuals_I_care:
    individual_id = all_individuals[individual]['id']

    # take the temporary inp file and run the simulation using epanet     
    net = wntr.epanet.io.WaterNetworkModel(f'.tmp/{individual_id}.inp')
    #net.options.time.hydraulic_timestep = 1*60*60
    #net.options.time.report_timestep = 60*60
    net.options.time.report_timestep = net.options.time.hydraulic_timestep
    sim = wntr.sim.EpanetSimulator(net)
    results_int[f'{individual_id}'] = sim.run_sim()
    results = results_int[f'{individual_id}']

    fig = make_subplots(rows=3, cols=1, subplot_titles=('Head', 'Demand', 'Pumping Station Flowrate'), shared_xaxes=True, vertical_spacing=0.03)
    # fig.add_trace(go.Scatter(x=results.node['head'].index/3600, y=results.node['head'].loc[:,'T0'], mode='lines', name='T0'), row=1, col=1)
    fig.add_trace(go.Scatter(x=results.node['head'].index/3600, y=results.node['head'].loc[:,'41'], mode='lines', name='41'), row=1, col=1)
    fig.add_trace(go.Scatter(x=results.node['head'].index/3600, y=results.node['head'].loc[:,'21'], mode='lines', name='21'), row=1, col=1)
    fig.add_trace(go.Scatter(x=results.node['head'].index/3600, y=results.node['head'].loc[:,'14'], mode='lines', name='14'), row=1, col=1)
    fig.add_trace(go.Scatter(x=results.node['head'].index/3600, y=results.node['head'].loc[:,'42'], mode='lines', name='42'), row=1, col=1)
    fig.add_trace(go.Scatter(x=results.node['head'].index/3600, y=results.node['head'].loc[:,'22'], mode='lines', name='22'), row=1, col=1)
    fig.add_trace(go.Scatter(x=results.node['head'].index/3600, y=results.node['head'].loc[:,'17'], mode='lines', name='17'), row=1, col=1)

    #fig.add_trace(go.Scatter(x=results.node['demand'].index/3600, y=results.node['demand'].loc[:,'T0'], mode='lines', name='T0'), row=2, col=1)
    fig.add_trace(go.Scatter(x=results.node['demand'].index/3600, y=results.node['demand'].loc[:,'41'], mode='lines', name='41'), row=2, col=1)
    fig.add_trace(go.Scatter(x=results.node['demand'].index/3600, y=results.node['demand'].loc[:,'21'], mode='lines', name='21'), row=2, col=1)
    fig.add_trace(go.Scatter(x=results.node['demand'].index/3600, y=results.node['demand'].loc[:,'14'], mode='lines', name='14'), row=2, col=1)
    fig.add_trace(go.Scatter(x=results.node['demand'].index/3600, y=results.node['demand'].loc[:,'42'], mode='lines', name='42'), row=2, col=1)
    fig.add_trace(go.Scatter(x=results.node['demand'].index/3600, y=results.node['demand'].loc[:,'22'], mode='lines', name='22'), row=2, col=1)
    fig.add_trace(go.Scatter(x=results.node['demand'].index/3600, y=results.node['demand'].loc[:,'17'], mode='lines', name='17'), row=2, col=1)

    fig.add_trace(go.Scatter(x=results.link['flowrate'].index/3600, y=results.link['flowrate'].loc[:,'78']+results.link['flowrate'].loc[:,'79']+results.link['flowrate'].loc[:,'80'], 
                             mode='lines', name='Pstat')
                             , row=3, col=1)
    fig.update_layout(title_text=f'Individual {individual_id}', height=figspace_in[1]*dpi*2)
    fig.show()
    

    # build whatever plot you want :)

    


In [ ]:
# I need a way to click on a point and see the individual that generated that point
import subprocess
beme_folder='../'
e = 2 # experiment to use
individuals_I_care = [858, 141, 872]
all_individuals = [ind for island in exps[e]['archipelago']['islands'] for ind in island['generations'][-1]['individuals'] ]

filter_problem_settings(os.path.join(exps[e]['folder'], 'bemeopt__settings.json'),
                        folder=exps[e]['folder'])

for individual in individuals_I_care:
    individual_id = all_individuals[individual]['id']
    save_individual_json(all_individuals[individual])

    # run the simulation of beme to save the inp file from shell
    command=f'{beme_folder}build-vs-code/cli/beme-sim .tmp/bemesim__settings.json .tmp/bemesim__{individual_id}__dv.json --saveinp'
    subprocess.run(command, shell=True, check=True)

subprocess.run('mv *.inp .tmp/', shell=True, check=True)

In [ ]:
results_pd = {} 
for individual in individuals_I_care:
    individual_id = all_individuals[individual]['id']

    # take the temporary inp file and run the simulation using epanet     
    net = wntr.epanet.io.WaterNetworkModel(f'.tmp/{individual_id}.inp')
    sim = wntr.sim.EpanetSimulator(net)
    net.options.time.report_timestep = net.options.time.hydraulic_timestep
    results_pd[f'{individual_id}'] = sim.run_sim()
    results = results_pd[f'{individual_id}']

    fig = make_subplots(rows=3, cols=1, subplot_titles=('Head', 'Demand', 'Pumping Station Flowrate'), shared_xaxes=True, vertical_spacing=0.03)
    fig.add_trace(go.Scatter(x=results.node['head'].index/3600, y=results.node['head'].loc[:,'41'], mode='lines', name='41'), row=1, col=1)
    fig.add_trace(go.Scatter(x=results.node['head'].index/3600, y=results.node['head'].loc[:,'21'], mode='lines', name='21'), row=1, col=1)
    fig.add_trace(go.Scatter(x=results.node['head'].index/3600, y=results.node['head'].loc[:,'14'], mode='lines', name='14'), row=1, col=1)
    fig.add_trace(go.Scatter(x=results.node['head'].index/3600, y=results.node['head'].loc[:,'42'], mode='lines', name='42'), row=1, col=1)
    fig.add_trace(go.Scatter(x=results.node['head'].index/3600, y=results.node['head'].loc[:,'22'], mode='lines', name='22'), row=1, col=1)
    fig.add_trace(go.Scatter(x=results.node['head'].index/3600, y=results.node['head'].loc[:,'17'], mode='lines', name='17'), row=1, col=1)

    fig.add_trace(go.Scatter(x=results.node['demand'].index/3600, y=results.node['demand'].loc[:,'41'], mode='lines', name='41'), row=2, col=1)
    fig.add_trace(go.Scatter(x=results.node['demand'].index/3600, y=results.node['demand'].loc[:,'21'], mode='lines', name='21'), row=2, col=1)
    fig.add_trace(go.Scatter(x=results.node['demand'].index/3600, y=results.node['demand'].loc[:,'14'], mode='lines', name='14'), row=2, col=1)
    fig.add_trace(go.Scatter(x=results.node['demand'].index/3600, y=results.node['demand'].loc[:,'42'], mode='lines', name='42'), row=2, col=1)
    fig.add_trace(go.Scatter(x=results.node['demand'].index/3600, y=results.node['demand'].loc[:,'22'], mode='lines', name='22'), row=2, col=1)
    fig.add_trace(go.Scatter(x=results.node['demand'].index/3600, y=results.node['demand'].loc[:,'17'], mode='lines', name='17'), row=2, col=1)

    fig.add_trace(go.Scatter(x=results.link['flowrate'].index/3600, y=results.link['flowrate'].loc[:,'78']+results.link['flowrate'].loc[:,'79']+results.link['flowrate'].loc[:,'80'], 
                             mode='lines', name='Pstat')
                             , row=3, col=1)
    fig.update_layout(title_text=f'Individual {individual_id}', height=figspace_in[1]*dpi*2)
    fig.show()

    # build whatever plot you want :)

In [ ]:
clear_space()